In [ ]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
all_data = pd.read_csv("HDFCBANK.csv")
all_data.head(5)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,1/3/2000,HDFCBANK,EQ,157.40,166.00,170.00,166.00,170.00,170.00,169.52,33259,5.638122e+11,NaN,NaN,NaN
1,1/4/2000,HDFCBANK,EQ,170.00,182.00,183.45,171.00,174.00,173.80,174.99,168710,2.952261e+12,NaN,NaN,NaN
2,1/5/2000,HDFCBANK,EQ,173.80,170.00,173.90,165.00,168.00,166.95,169.20,159820,2.704094e+12,NaN,NaN,NaN
3,1/6/2000,HDFCBANK,EQ,166.95,168.00,170.00,165.30,168.95,168.30,168.44,85026,1.432166e+12,NaN,NaN,NaN
4,1/7/2000,HDFCBANK,EQ,168.30,162.15,171.00,162.15,170.75,168.35,166.79,85144,1.420158e+12,NaN,NaN,NaN


In [ ]:
all_data=all_data.drop(['Date','Trades','Deliverable Volume','%Deliverble','Symbol','Series'],axis=1)
all_data['Volume_lag_1d'] = all_data['Volume'].shift(1)
#all_data['Turnover_lag_1d'] = all_data['Volume'].shift(1)
all_data.dropna(inplace=True)
x=all_data.drop(['Close','Volume','Turnover'],axis=1)
x.head()

,Prev Close,Open,High,Low,Last,VWAP,Volume_lag_1d
1,170.00,182.00,183.45,171.00,174.00,174.99,33259.0
2,173.80,170.00,173.90,165.00,168.00,169.20,168710.0
3,166.95,168.00,170.00,165.30,168.95,168.44,159820.0
4,168.30,162.15,171.00,162.15,170.75,166.79,85026.0
5,168.35,172.90,179.50,165.00,166.30,167.79,85144.0


In [ ]:
y=all_data['Close']
y.head()

1    173.80
2    166.95
3    168.30
4    168.35
5    165.90
Name: Close, dtype: float64

In [ ]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
scld=MinMaxScaler(feature_range=(0,1))
arr_scld=scld.fit_transform(x)
x_scld=pd.DataFrame(arr_scld,columns=x.columns)
x_scld.head()
x_scld.describe()

,Prev Close,Open,High,Low,Last,VWAP,Volume_lag_1d
0,0.002747,0.008258,0.006438,0.005841,0.004583,0.005641,0.000320
1,0.004329,0.003266,0.002484,0.003338,0.002083,0.003237,0.001667
2,0.001478,0.002434,0.000869,0.003463,0.002479,0.002922,0.001579
3,0.002040,0.000000,0.001283,0.002149,0.003229,0.002237,0.000835
4,0.002060,0.004472,0.004803,0.003338,0.001375,0.002652,0.000836


,Prev Close,Open,High,Low,Last,VWAP,Volume_lag_1d
count,5305.000000,5305.000000,5305.000000,5305.000000,5305.000000,5305.000000,5305.000000
mean,0.351255,0.351720,0.352839,0.349221,0.351884,0.351009,0.020868
std,0.264615,0.264333,0.265546,0.262636,0.264866,0.263744,0.039425
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.131764,0.133369,0.132090,0.132015,0.132500,0.132300,0.003030
50%,0.321179,0.323460,0.325453,0.319272,0.322167,0.322114,0.009380
75%,0.523539,0.524929,0.526662,0.518213,0.525000,0.522716,0.021086
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_x,test_x,train_y,test_y=train_test_split(x_scld,y,test_size=0.20,random_state=1)
train_x.shape
test_x.shape
train_y.shape
test_y.shape

(4244, 7)

(1061, 7)

(4244,)

(1061,)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=44)
model.fit(train_x, train_y)
predicted_values=model.predict(test_x)
predicted_values


DecisionTreeRegressor(random_state=44)

array([1159.6 , 1792.3 , 1244.65, ...,  239.55,  941.5 ,  680.55])

In [ ]:
#visualize the decision tree graph
from sklearn import tree
import graphviz
with open('tree.dot','w') as dotfile:
  tree.export_graphviz(model,out_file=dotfile,feature_names=train_x.columns,filled=True)
  dotfile.close()

In [ ]:
from graphviz import Source
with open('tree.dot','r') as f:
  text=f.read()
  plot=Source(text)
  plot

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#we will also split our data into train and test datasets and Build the model on train dataset and apply the model on test datset.
from sklearn import tree
import pydotplus
from sklearn.model_selection import GridSearchCV
my_max_depth=[2,5,10,15,20,25]
my_criterion=["squared_error", "friedman_mse", "absolute_error","poisson"]
#BUILDING THE MODEL
mytreeone=DecisionTreeRegressor()
#CROSS VALIDATION TELLS HOW A MODEL PERFORMS ON A DATASET USING MULTIPLE SAMPLES OF TRAIN DATA
model=GridSearchCV(estimator=mytreeone,cv=3,param_grid=dict(max_depth=my_max_depth,criterion=my_criterion))

In [ ]:
model.fit(train_x, train_y)
predicted_values=model.predict(test_x)
predicted_values

GridSearchCV(cv=3, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [2, 5, 10, 15, 20, 25]})

array([1159.6 , 1791.05, 1246.2 , ...,  239.6 ,  935.  ,  680.95])

**STEP6-EVALUATE MODEL PERFORMANCE for GridSearchCV**

**MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_absolute_error
print("MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(mean_absolute_error(predicted_values,test_y),3)

MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


3.218

**MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_squared_error
print("MEAN SQUARE ERROR (MSE) FOR TEST DATA IS")
mse=np.round(mean_squared_error(predicted_values,test_y),3)
mse

MEAN SQUARE ERROR (MSE) FOR TEST DATA IS


23.16

**ROOT MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
import math
math.sqrt(mse)

4.812483766206386

**MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import median_absolute_error
print("MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(median_absolute_error(predicted_values,test_y),3)

MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


2.1

**Mean Absolute Percentage Error FOR TEST DATA**

In [ ]:
print("MAPE FOR TEST DATA IS")
mape1 = np.mean(np.abs((test_y - predicted_values)/test_y))*100
np.round(mape1,3)

MAPE FOR TEST DATA IS


0.378

In [ ]:
n_estimators=[10,500,100,2000]
criterion=["squared_error", "absolute_error", "poisson"]   
min_samples_split=[2,100,20,50,100]
max_depth=[20,30,40,50]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid={'n_estimators':n_estimators,'criterion':criterion,'min_samples_split':min_samples_split,'max_depth':max_depth}
rf=RandomForestRegressor()
model=RandomizedSearchCV(rf,param_grid,cv=5,verbose=2,n_jobs=1)

In [ ]:
model.fit(train_x, train_y)
predicted_values=model.predict(test_x)
predicted_values

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END criterion=absolute_error, max_depth=20, min_samples_split=2, n_estimators=10; total time=   1.8s
[CV] END criterion=absolute_error, max_depth=20, min_samples_split=2, n_estimators=10; total time=   1.8s
[CV] END criterion=absolute_error, max_depth=20, min_samples_split=2, n_estimators=10; total time=   1.8s
[CV] END criterion=absolute_error, max_depth=20, min_samples_split=2, n_estimators=10; total time=   1.8s
[CV] END criterion=absolute_error, max_depth=20, min_samples_split=2, n_estimators=10; total time=   1.8s
[CV] END criterion=absolute_error, max_depth=40, min_samples_split=2, n_estimators=500; total time= 1.5min
[CV] END criterion=absolute_error, max_depth=40, min_samples_split=2, n_estimators=500; total time= 1.7min
[CV] END criterion=absolute_error, max_depth=40, min_samples_split=2, n_estimators=500; total time= 1.5min
[CV] END criterion=absolute_error, max_depth=40, min_samples_split=2, n_estimators=500; 

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=1,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'poisson'],
                                        'max_depth': [20, 30, 40, 50],
                                        'min_samples_split': [2, 100, 20, 50,
                                                              100],
                                        'n_estimators': [10, 500, 100, 2000]},
                   verbose=2)

array([1154.8316 , 1791.2168 , 1249.795  , ...,  239.3345 ,  936.4274 ,
        680.10335])

**STEP6-EVALUATE MODEL PERFORMANCE for RandomizedSearchCV**

**MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_absolute_error
print("MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(mean_absolute_error(predicted_values,test_y),3)

MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


2.447

**MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_squared_error
print("MEAN SQUARE ERROR (MSE) FOR TEST DATA IS")
mse=np.round(mean_squared_error(predicted_values,test_y),3)
mse

MEAN SQUARE ERROR (MSE) FOR TEST DATA IS


15.249

**ROOT MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
import math
math.sqrt(mse)

3.90499679897436

**MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import median_absolute_error
print("MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(median_absolute_error(predicted_values,test_y),3)

MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


1.491

**Mean Absolute Percentage Error FOR TEST DATA**

In [ ]:
print("MAPE FOR TEST DATA IS")
mape1 = np.mean(np.abs((test_y - predicted_values)/test_y))*100
np.round(mape1,3)

MAPE FOR TEST DATA IS


0.288

In [ ]:
import xgboost as xg

In [ ]:
model = xg.XGBRegressor(objective ='reg:linear',seed = 123,booster='gbtree',max_depth=5,n_estimators=200,random_state=42)

In [ ]:
model.fit(train_x, train_y)
predicted_values=model.predict(test_x)
predicted_values

[17:19:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(max_depth=5, n_estimators=200, random_state=42, seed=123)

array([1150.8633 , 1788.5756 , 1249.9127 , ...,  237.47133,  935.37366,
        679.01385], dtype=float32)

**STEP6-EVALUATE MODEL PERFORMANCE for xgboost**

**MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_absolute_error
print("MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(mean_absolute_error(predicted_values,test_y),3)

MEAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


3.253

**MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import mean_squared_error
print("MEAN SQUARE ERROR (MSE) FOR TEST DATA IS")
mse=np.round(mean_squared_error(predicted_values,test_y),3)
mse

MEAN SQUARE ERROR (MSE) FOR TEST DATA IS


22.784

**ROOT MEAN SQUARE ERROR (MSE) FOR TEST DATA**

In [ ]:
import math
math.sqrt(mse)

4.773258844856415

**MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA**

In [ ]:
from sklearn.metrics import median_absolute_error
print("MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS")
np.round(median_absolute_error(predicted_values,test_y),3)

MEDIAN ABSOLUTE ERROR (MAE) FOR TEST DATA IS


2.115

**Mean Absolute Percentage Error FOR TEST DATA**

In [ ]:
print("MAPE FOR TEST DATA IS")
mape1 = np.mean(np.abs((test_y - predicted_values)/test_y))*100
np.round(mape1,3)

MAPE FOR TEST DATA IS


0.374

In [ ]:
!jupyter nbconvert StockMarketPrediction_DT_GRIDSEARCHCV_RF_XGBOOST.ipynb

[NbConvertApp] Converting notebook StockMarketPrediction_DT_GRIDSEARCHCV_RF_XGBOOST.ipynb to html
[NbConvertApp] Writing 9074314 bytes to StockMarketPrediction_DT_GRIDSEARCHCV_RF_XGBOOST.html
